# 데이터 불러오기

In [53]:
import pandas as pd
file_path = 'uci-secom.csv'
df = pd.read_csv(file_path)

print(df.head())

                  Time        0        1          2          3       4      5  \
0  2008-07-19 11:55:00  3030.93  2564.00  2187.7333  1411.1265  1.3602  100.0   
1  2008-07-19 12:32:00  3095.78  2465.14  2230.4222  1463.6606  0.8294  100.0   
2  2008-07-19 13:17:00  2932.61  2559.94  2186.4111  1698.0172  1.5102  100.0   
3  2008-07-19 14:43:00  2988.72  2479.90  2199.0333   909.7926  1.3204  100.0   
4  2008-07-19 15:22:00  3032.24  2502.87  2233.3667  1326.5200  1.5334  100.0   

          6       7       8  ...       581     582     583     584      585  \
0   97.6133  0.1242  1.5005  ...       NaN  0.5005  0.0118  0.0035   2.3630   
1  102.3433  0.1247  1.4966  ...  208.2045  0.5019  0.0223  0.0055   4.4447   
2   95.4878  0.1241  1.4436  ...   82.8602  0.4958  0.0157  0.0039   3.1745   
3  104.2367  0.1217  1.4882  ...   73.8432  0.4990  0.0103  0.0025   2.0544   
4  100.3967  0.1235  1.5031  ...       NaN  0.4800  0.4766  0.1045  99.3032   

      586     587     588       589  P

# 데이터 전처리

문제 해결할것
1. time :분석에 쓰기 애매 ->열 삭제하자
2. NaN -> 1차로 결측치 20%넘는 애들은 열 지우고, 2차로 평균값 넣어주자
3. pass/fail 목표인데 1,-1로 되어있음 -> 0 정상, 1 불량으로 바꿔주자

In [54]:
#1
print(df['Pass/Fail'].value_counts())

Pass/Fail
-1    1463
 1     104
Name: count, dtype: int64


In [55]:
df_processed = df.drop(['Time'], axis=1)

In [56]:
print(df_processed.head())

         0        1          2          3       4      5         6       7  \
0  3030.93  2564.00  2187.7333  1411.1265  1.3602  100.0   97.6133  0.1242   
1  3095.78  2465.14  2230.4222  1463.6606  0.8294  100.0  102.3433  0.1247   
2  2932.61  2559.94  2186.4111  1698.0172  1.5102  100.0   95.4878  0.1241   
3  2988.72  2479.90  2199.0333   909.7926  1.3204  100.0  104.2367  0.1217   
4  3032.24  2502.87  2233.3667  1326.5200  1.5334  100.0  100.3967  0.1235   

        8       9  ...       581     582     583     584      585     586  \
0  1.5005  0.0162  ...       NaN  0.5005  0.0118  0.0035   2.3630     NaN   
1  1.4966 -0.0005  ...  208.2045  0.5019  0.0223  0.0055   4.4447  0.0096   
2  1.4436  0.0041  ...   82.8602  0.4958  0.0157  0.0039   3.1745  0.0584   
3  1.4882 -0.0124  ...   73.8432  0.4990  0.0103  0.0025   2.0544  0.0202   
4  1.5031 -0.0031  ...       NaN  0.4800  0.4766  0.1045  99.3032  0.0202   

      587     588       589  Pass/Fail  
0     NaN     NaN       NaN

In [57]:
#2
missing_values = df_processed.isnull().sum()
print("결측치 개수")
print(missing_values)

결측치 개수
0             6
1             7
2            14
3            14
4            14
             ..
586           1
587           1
588           1
589           1
Pass/Fail     0
Length: 591, dtype: int64


In [74]:
print(f"총 데이터 수 {len(df_processed)}")
print(f"결측치 하나라도 있는 칼럼 수 {len(missing_values[missing_values > 0])}")

총 데이터 수 1567
결측치 하나라도 있는 칼럼 수 538


In [59]:
total_rows = len(df_processed)
threshold = total_rows * 0.2
cols_to_drop = missing_values[missing_values > threshold].index
print(f"20% 이상 결측치가 있는 칼럼 (총 {len(cols_to_drop)}개)")
print(cols_to_drop)

20% 이상 결측치가 있는 칼럼 (총 32개)
Index(['72', '73', '85', '109', '110', '111', '112', '157', '158', '220',
       '244', '245', '246', '247', '292', '293', '345', '346', '358', '382',
       '383', '384', '385', '492', '516', '517', '518', '519', '578', '579',
       '580', '581'],
      dtype='object')


In [75]:
df_cleaned = df_processed.drop(cols_to_drop, axis=1)
print(f"원본 칼럼 수 {len(df_processed.columns)}")
print(f"제거된 칼럼 수 {len(cols_to_drop)}")
print(f"남아있는 칼럼 수 {len(df_cleaned.columns)}")

원본 칼럼 수 591
제거된 칼럼 수 32
남아있는 칼럼 수 559


In [61]:
mean_values = df_cleaned.drop('Pass/Fail', axis=1).mean()
mean_values.head()

,0
0,3014.452896
1,2495.850231
2,2200.547318
3,1396.376627
4,4.197013


In [62]:
df_filled = df_cleaned.copy()
df_filled.update(df_cleaned.drop('Pass/Fail', axis=1).fillna(mean_values))

In [63]:
print(f"남아있는 결측치 수: {df_filled.isnull().sum().sum()}")

남아있는 결측치 수: 0


In [64]:
#3
df_final = df_filled.copy()
df_final['Target'] = df_final['Pass/Fail'].map({-1: 0, 1: 1})
df_final

,0,1,2,3,4,5,6,7,8,9,...,582,583,584,585,586,587,588,589,Pass/Fail,Target
0,3030.93,2564.00,2187.7333,1411.1265,1.3602,100.0,97.6133,0.1242,1.500500,0.016200,...,0.5005,0.0118,0.0035,2.3630,0.021458,0.016475,0.005283,99.670066,-1,0
1,3095.78,2465.14,2230.4222,1463.6606,0.8294,100.0,102.3433,0.1247,1.496600,-0.000500,...,0.5019,0.0223,0.0055,4.4447,0.009600,0.020100,0.006000,208.204500,-1,0
2,2932.61,2559.94,2186.4111,1698.0172,1.5102,100.0,95.4878,0.1241,1.443600,0.004100,...,0.4958,0.0157,0.0039,3.1745,0.058400,0.048400,0.014800,82.860200,1,1
3,2988.72,2479.90,2199.0333,909.7926,1.3204,100.0,104.2367,0.1217,1.488200,-0.012400,...,0.4990,0.0103,0.0025,2.0544,0.020200,0.014900,0.004400,73.843200,-1,0
4,3032.24,2502.87,2233.3667,1326.5200,1.5334,100.0,100.3967,0.1235,1.503100,-0.003100,...,0.4800,0.4766,0.1045,99.3032,0.020200,0.014900,0.004400,73.843200,-1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1562,2899.41,2464.36,2179.7333,3085.3781,1.4843,100.0,82.2467,0.1248,1.342400,-0.004500,...,0.4988,0.0143,0.0039,2.8669,0.006800,0.013800,0.004700,203.172000,-1,0
1563,3052.31,2522.55,2198.5667,1124.6595,0.8763,100.0,98.4689,0.1205,1.433300,-0.006100,...,0.4975,0.0131,0.0036,2.6238,0.006800,0.013800,0.004700,203.172000,-1,0
1564,2978.81,2379.78,2206.3000,1110.4967,0.8236,100.0,99.4122,0.1208,1.462862,-0.000841,...,0.4987,0.0153,0.0041,3.0590,0.019700,0.008600,0.002500,43.523100,-1,0
1565,2894.92,2532.01,2177.0333,1183.7287,1.5726,100.0,98.7978,0.1213,1.462200,-0.007200,...,0.5004,0.0178,0.0038,3.5662,0.026200,0.024500,0.007500,93.494100,-1,0


In [65]:
df_final = df_final.drop('Pass/Fail', axis=1)
print(df_final['Target'].value_counts())

Target
0    1463
1     104
Name: count, dtype: int64


# 모델링

In [66]:
from sklearn.ensemble import RandomForestClassifier
import numpy as np

In [67]:
x = df_final.drop('Target', axis=1)
y = df_final['Target']

In [68]:
model = RandomForestClassifier(n_estimators=100,
                             class_weight='balanced',
                             random_state=42)

In [69]:
model.fit(x, y)

RandomForestClassifier(class_weight='balanced', random_state=42)

In [70]:
importances = model.feature_importances_

In [71]:
feature_importance_df = pd.DataFrame({
    'Feature': x.columns,
    'Importance': importances
}).sort_values(by='Importance', ascending=False)

In [73]:
print("수율 불량에 영향 미치는 변수 top10")
print(feature_importance_df.head(10))

수율 불량에 영향 미치는 변수 top10
    Feature  Importance
59       59    0.022450
100     103    0.013142
330     348    0.010711
123     130    0.009493
33       33    0.008905
65       65    0.007034
64       64    0.006951
114     121    0.006809
196     205    0.006654
122     129    0.006576
